In [2]:
from edc import check_compatibility
check_compatibility("user-2022.10-14", dependencies=["GEODB"])



---------

The following environment variables are available:

* `GEODB_AUTH_AUD`, `GEODB_AUTH_CLIENT_ID`, `GEODB_AUTH_DOMAIN`, `GEODB_API_SERVER_URL`, `GEODB_AUTH_CLIENT_SECRET`, `GEODB_API_SERVER_PORT`


## Manage Collections in your GeoDB



### Connecting to the GeoDB

In [3]:
from xcube_geodb.core.geodb import GeoDBClient
from xcube_geodb.version import version

In [4]:
geodb = GeoDBClient()

In [5]:
# If you are logged in, this will tell you what account the system currently uses
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

Checkout the deployed SQL version:

In [6]:
geodb.get_geodb_sql_version()

'v1.0.4'

Checkout the version of xcube geoDB software in your workspace: 

In [7]:
version

'1.0.4'

In [8]:
# Lets get already existing collections
ds = geodb.get_my_collections()
ds


,owner,database,collection
0,geodb_49a05d04-5d72-4c0f-9065-6e6827fd1871,anja,E1
1,geodb_49a05d04-5d72-4c0f-9065-6e6827fd1871,anja,E10a1
2,geodb_49a05d04-5d72-4c0f-9065-6e6827fd1871,anja,E10a2
3,geodb_49a05d04-5d72-4c0f-9065-6e6827fd1871,anja,E11
4,geodb_49a05d04-5d72-4c0f-9065-6e6827fd1871,anja,E1a
...,...,...,...
912,geodb_cdcbc7da-dc26-42dc-bff9-d4336f98dca0,polar,polar_sea_ice
913,geodb_cdcbc7da-dc26-42dc-bff9-d4336f98dca0,polar,polar_sea_ice_3413
914,geodb_admin,public,land_use
915,geodb_d2c4722a-cc19-4ec1-b575-0cdb6876d4a7,test_duplicate,test


### Creating collections

Once the connection has been established you will be able to create a collection. The collection will contain standard properties (fields) plus custom properties
which you can add at your disgretion. Please use [PostGreSQL type definitions](https://www.postgresql.org/docs/11/datatype.html). We recommend stying simple with
your data types as we have not tested every single type.

In [9]:
ds = geodb.get_my_collections()

if ds[(ds.database == geodb.whoami) & (ds.collection == 'land_use')].collection.count() == 0:
    # Have a look at fiona feature schema
    collections = {
            "land_use": 
            {
                "crs": 3794,
                "properties": 
                {
                    "RABA_PID": "float", 
                    "RABA_ID": "float", 
                    "D_OD": "date"
                }
            }
        }


    geodb.create_collections(collections)
    import geopandas
    import os
    gdf = geopandas.read_file(os.path.expanduser("~/.shared/notebooks/eurodatacube/notebooks/curated/data/sample/land_use.shp"))
    geodb.insert_into_collection('land_use', gdf)

In [10]:
ds = geodb.get_my_collections(database=geodb.whoami)
ds

,owner,database,collection
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2021-10-04-test-1234
1,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2021-10-04-test-2
2,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2021-10-04-test-3
3,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2021-10-04-test-7
4,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2021-10-04-test-9876
5,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2021-10-04-test-coll
6,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,alster
7,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use


See the chanhes made to the collection: 

In [11]:
geodb.get_event_log('land_use')

,event_type,message,username,date
0,added rows,9827 rows inserted into geodb_418dfeac-15f0-46...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:03.607382
1,dropped rows,from collection geodb_418dfeac-15f0-4606-9edb-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:04.126797
2,added property,"{name: test_prop, type: integer} to collection...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:06.172545
3,dropped property,test_prop from collection geodb_418dfeac-15f0-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.069597
4,added property,"{name: test1, type: integer} to collection geo...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.337766
5,added property,"{name: test2, type: date} to collection geodb_...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.39186
6,dropped property,test1 from collection geodb_418dfeac-15f0-4606...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.11931
7,dropped property,test2 from collection geodb_418dfeac-15f0-4606...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.175558
8,dropped,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.450506
9,created,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:45.888491


### Loading data into a dataset

Once the table has been created, you can load data into the dataset. The example below loads a shapefile. The attributes of the shapefile correspond to the dataset's properties.


In [12]:
import geopandas
import os
gdf = geopandas.read_file(os.path.expanduser("~/.shared/notebooks/eurodatacube/notebooks/curated/data/sample/land_use.shp"))
gdf

,RABA_PID,RABA_ID,D_OD,geometry
0,4770326.0,1410,2019-03-26,"POLYGON ((453952.629 91124.177, 453952.696 911..."
1,4770325.0,1300,2019-03-26,"POLYGON ((453810.376 91150.199, 453812.552 911..."
2,2305689.0,7000,2019-02-25,"POLYGON ((456099.635 97696.070, 456112.810 976..."
3,2305596.0,1100,2019-02-25,"POLYGON ((455929.405 97963.785, 455933.284 979..."
4,2310160.0,1100,2019-03-11,"POLYGON ((461561.512 96119.256, 461632.114 960..."
...,...,...,...,...
9822,6253989.0,1600,2019-03-08,"POLYGON ((460637.334 96865.891, 460647.927 969..."
9823,6252044.0,1600,2019-03-26,"POLYGON ((459467.868 96839.686, 459467.770 968..."
9824,6245985.0,2000,2019-04-08,"POLYGON ((459488.998 94066.248, 459498.145 940..."
9825,6245986.0,2000,2019-02-20,"POLYGON ((459676.680 94000.000, 459672.469 939..."


In [13]:
geodb.insert_into_collection('land_use', gdf)

Processing rows from 0 to 999


Processing rows from 1000 to 1999


Processing rows from 2000 to 2999


Processing rows from 3000 to 3999


Processing rows from 4000 to 4999


Processing rows from 5000 to 5999


Processing rows from 6000 to 6999


Processing rows from 7000 to 7999


Processing rows from 8000 to 8999


Processing rows from 9000 to 9827


See the chanhes made to the collection: 

In [14]:
geodb.get_event_log('land_use')

,event_type,message,username,date
0,added rows,9827 rows inserted into geodb_418dfeac-15f0-46...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:03.607382
1,dropped rows,from collection geodb_418dfeac-15f0-4606-9edb-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:04.126797
2,added property,"{name: test_prop, type: integer} to collection...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:06.172545
3,dropped property,test_prop from collection geodb_418dfeac-15f0-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.069597
4,added property,"{name: test1, type: integer} to collection geo...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.337766
5,added property,"{name: test2, type: date} to collection geodb_...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.39186
6,dropped property,test1 from collection geodb_418dfeac-15f0-4606...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.11931
7,dropped property,test2 from collection geodb_418dfeac-15f0-4606...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.175558
8,dropped,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.450506
9,created,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:45.888491


In [15]:
geodb.get_collection('land_use', query="raba_id=eq.7000")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,3,2022-10-13T11:29:34.046006+00:00,None,"POLYGON ((456099.635 97696.070, 456112.810 976...",2305689,7000,2019-02-25
1,26,2022-10-13T11:29:34.046006+00:00,None,"POLYGON ((459898.930 100306.841, 459906.288 10...",2301992,7000,2019-04-06
2,95,2022-10-13T11:29:34.046006+00:00,None,"POLYGON ((459591.248 92619.056, 459592.745 926...",2333229,7000,2019-02-20
3,115,2022-10-13T11:29:34.046006+00:00,None,"POLYGON ((459013.303 100354.458, 459022.756 10...",2336738,7000,2019-03-15
4,129,2022-10-13T11:29:34.046006+00:00,None,"POLYGON ((460851.200 93442.039, 460846.405 934...",6292562,7000,2019-02-21
...,...,...,...,...,...,...,...
763,19246,2022-10-13T12:40:17.519409+00:00,None,"POLYGON ((456523.670 94000.000, 456550.368 939...",6187993,7000,2019-03-13
764,19373,2022-10-13T12:40:17.519409+00:00,None,"POLYGON ((460793.279 94000.000, 460797.466 939...",6219403,7000,2019-02-21
765,19387,2022-10-13T12:40:17.519409+00:00,None,"POLYGON ((461750.000 98434.292, 461750.000 984...",6219602,7000,2019-03-20
766,19451,2022-10-13T12:40:17.519409+00:00,None,"POLYGON ((463628.912 97501.218, 463636.859 975...",6215237,7000,2019-03-20


### Delete from a Collection

In [16]:
geodb.delete_from_collection('land_use', query="raba_id=eq.7000")

See the chanhes made to the collection: 

In [17]:
geodb.get_event_log('land_use')

,event_type,message,username,date
0,added rows,9827 rows inserted into geodb_418dfeac-15f0-46...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:03.607382
1,dropped rows,from collection geodb_418dfeac-15f0-4606-9edb-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:04.126797
2,added property,"{name: test_prop, type: integer} to collection...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:06.172545
3,dropped property,test_prop from collection geodb_418dfeac-15f0-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.069597
4,added property,"{name: test1, type: integer} to collection geo...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.337766
5,added property,"{name: test2, type: date} to collection geodb_...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.39186
6,dropped property,test1 from collection geodb_418dfeac-15f0-4606...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.11931
7,dropped property,test2 from collection geodb_418dfeac-15f0-4606...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.175558
8,dropped,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.450506
9,created,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:45.888491


In [18]:
geodb.get_collection('land_use', query="raba_id=eq.7000")

,Empty Result


### Updating a Collection

In [19]:
geodb.get_collection('land_use', query="raba_id=eq.1300")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,2,2022-10-13T11:29:34.046006+00:00,None,"POLYGON ((453810.376 91150.199, 453812.552 911...",4770325,1300,2019-03-26
1,10,2022-10-13T11:29:34.046006+00:00,None,"POLYGON ((456547.427 91543.640, 456544.255 915...",2318555,1300,2019-03-14
2,63,2022-10-13T11:29:34.046006+00:00,None,"POLYGON ((456201.531 98685.274, 456199.109 986...",2304287,1300,2019-02-25
3,86,2022-10-13T11:29:34.046006+00:00,None,"POLYGON ((454709.766 97354.278, 454704.878 973...",2331038,1300,2019-01-05
4,87,2022-10-13T11:29:34.046006+00:00,None,"POLYGON ((453820.737 98574.017, 453816.740 985...",2357574,1300,2019-01-16
...,...,...,...,...,...,...,...
1785,19593,2022-10-13T12:40:17.519409+00:00,None,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2019-03-04
1786,19594,2022-10-13T12:40:17.519409+00:00,None,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2019-04-01
1787,19625,2022-10-13T12:40:17.519409+00:00,None,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2019-01-05
1788,19626,2022-10-13T12:40:17.519409+00:00,None,"POLYGON ((461378.845 91124.318, 461490.224 911...",6264865,1300,2019-02-21


In [20]:
geodb.update_collection('land_use', query="raba_id=eq.1300", values={'d_od': '2000-01-01'})

In [21]:
geodb.get_collection('land_use', query="raba_id=eq.1300")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,10,2022-10-13T11:29:34.046006+00:00,2022-10-13T12:40:19.645933+00:00,"POLYGON ((456547.427 91543.640, 456544.255 915...",2318555,1300,2000-01-01
1,103,2022-10-13T11:29:34.046006+00:00,2022-10-13T12:40:19.645933+00:00,"POLYGON ((456100.880 96973.323, 456111.084 969...",2332420,1300,2000-01-01
2,108,2022-10-13T11:29:34.046006+00:00,2022-10-13T12:40:19.645933+00:00,"POLYGON ((454070.991 90528.946, 454064.259 905...",4674624,1300,2000-01-01
3,117,2022-10-13T11:29:34.046006+00:00,2022-10-13T12:40:19.645933+00:00,"POLYGON ((454637.692 96679.827, 454630.584 966...",2336101,1300,2000-01-01
4,136,2022-10-13T11:29:34.046006+00:00,2022-10-13T12:40:19.645933+00:00,"POLYGON ((462781.112 100486.430, 462803.178 10...",2334416,1300,2000-01-01
...,...,...,...,...,...,...,...
1785,19592,2022-10-13T12:40:17.519409+00:00,2022-10-13T12:40:19.645933+00:00,"POLYGON ((458304.555 100146.103, 458306.726 10...",6268058,1300,2000-01-01
1786,19593,2022-10-13T12:40:17.519409+00:00,2022-10-13T12:40:19.645933+00:00,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2000-01-01
1787,19594,2022-10-13T12:40:17.519409+00:00,2022-10-13T12:40:19.645933+00:00,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2000-01-01
1788,19625,2022-10-13T12:40:17.519409+00:00,2022-10-13T12:40:19.645933+00:00,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2000-01-01


### Managing Properties of a Collection

In [22]:
geodb.get_my_collections()

,owner,database,collection
0,geodb_49a05d04-5d72-4c0f-9065-6e6827fd1871,anja,E1
1,geodb_49a05d04-5d72-4c0f-9065-6e6827fd1871,anja,E10a1
2,geodb_49a05d04-5d72-4c0f-9065-6e6827fd1871,anja,E10a2
3,geodb_49a05d04-5d72-4c0f-9065-6e6827fd1871,anja,E11
4,geodb_49a05d04-5d72-4c0f-9065-6e6827fd1871,anja,E1a
...,...,...,...
912,geodb_cdcbc7da-dc26-42dc-bff9-d4336f98dca0,polar,polar_sea_ice
913,geodb_cdcbc7da-dc26-42dc-bff9-d4336f98dca0,polar,polar_sea_ice_3413
914,geodb_admin,public,land_use
915,geodb_d2c4722a-cc19-4ec1-b575-0cdb6876d4a7,test_duplicate,test


In [23]:
geodb.get_properties('land_use')

,database,collection,column_name,data_type
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,id,integer
1,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,created_at,timestamp with time zone
2,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,modified_at,timestamp with time zone
3,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,raba_pid,double precision
4,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,raba_id,double precision
5,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,d_od,date
6,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,geometry,USER-DEFINED


In [24]:
geodb.add_property('land_use', "test_prop", 'integer')

In [25]:
geodb.get_properties('land_use')

,database,collection,column_name,data_type
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,id,integer
1,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,created_at,timestamp with time zone
2,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,modified_at,timestamp with time zone
3,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,raba_pid,double precision
4,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,raba_id,double precision
5,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,d_od,date
6,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,test_prop,integer
7,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,geometry,USER-DEFINED


In [26]:
geodb.drop_property('land_use', 'test_prop')

In [27]:
geodb.get_properties('land_use')

,database,collection,column_name,data_type
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,id,integer
1,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,created_at,timestamp with time zone
2,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,modified_at,timestamp with time zone
3,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,raba_pid,double precision
4,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,raba_id,double precision
5,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,d_od,date
6,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,geometry,USER-DEFINED


In [28]:
geodb.add_properties('land_use', properties={'test1': 'integer', 'test2': 'date'})

In [29]:
geodb.get_properties('land_use')

,database,collection,column_name,data_type
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,id,integer
1,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,created_at,timestamp with time zone
2,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,modified_at,timestamp with time zone
3,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,raba_pid,double precision
4,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,raba_id,double precision
5,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,d_od,date
6,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,test1,integer
7,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,test2,date
8,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,geometry,USER-DEFINED


In [30]:
geodb.drop_properties('land_use', properties=['test1', 'test2'])

In [31]:
geodb.get_properties('land_use')

,database,collection,column_name,data_type
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,id,integer
1,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,created_at,timestamp with time zone
2,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,modified_at,timestamp with time zone
3,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,raba_pid,double precision
4,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,raba_id,double precision
5,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,d_od,date
6,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,geometry,USER-DEFINED


In [32]:
geodb.drop_collection('land_use')

See the chanhes made to the collection: 

In [33]:
geodb.get_event_log('land_use')

,event_type,message,username,date
0,added rows,9827 rows inserted into geodb_418dfeac-15f0-46...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:03.607382
1,dropped rows,from collection geodb_418dfeac-15f0-4606-9edb-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:04.126797
2,added property,"{name: test_prop, type: integer} to collection...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:06.172545
3,dropped property,test_prop from collection geodb_418dfeac-15f0-...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.069597
4,added property,"{name: test1, type: integer} to collection geo...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.337766
5,added property,"{name: test2, type: date} to collection geodb_...",geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:09.39186
6,dropped property,test1 from collection geodb_418dfeac-15f0-4606...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.11931
7,dropped property,test2 from collection geodb_418dfeac-15f0-4606...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.175558
8,dropped,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:12.450506
9,created,collection geodb_418dfeac-15f0-4606-9edb-fd9eb...,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,2022-09-27T08:56:45.888491
